<a href="https://colab.research.google.com/github/rahulgundala007/NLP_text_summarization/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets torch


In [2]:
pip install accelerate -U

In [7]:
from datasets import load_dataset

dataset = load_dataset('Rahulgundala007/ToSdataset2')


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
from transformers import AutoTokenizer

model_name = 't5-small'  # You can choose other versions of T5 like 't5-base', 't5-large', etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    # Prefix input text with 'summarize: '
    inputs = ["summarize: " + doc for doc in examples['ToS_Detail']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # No prefix for summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['ToS_Summary'], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/65 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [9]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    save_strategy="epoch",           # Save the model at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    weight_decay=0.01,               # Weight decay for regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],   # Use your dataset splits appropriately
    eval_dataset=dataset['validation'] # Use your dataset splits appropriately
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.671835
2,No log,2.648273
3,No log,2.296098


TrainOutput(global_step=51, training_loss=4.88160915000766, metrics={'train_runtime': 605.114, 'train_samples_per_second': 0.322, 'train_steps_per_second': 0.084, 'total_flos': 26391651287040.0, 'train_loss': 4.88160915000766, 'epoch': 3.0})

In [10]:
model.save_pretrained('./my_t5_summarizer')
tokenizer.save_pretrained('./my_t5_summarizer')


('./my_t5_summarizer/tokenizer_config.json',
 './my_t5_summarizer/special_tokens_map.json',
 './my_t5_summarizer/spiece.model',
 './my_t5_summarizer/added_tokens.json',
 './my_t5_summarizer/tokenizer.json')

**Epochs:** The training ran for 3 epochs. An epoch is a full pass through your entire training dataset.

**Training Loss**: This indicates how well the model is fitting the training data. Lower values mean better fit. "No log" suggests that the loss wasn't logged during the training but only at the end of each epoch for validation.

**Validation Loss**: Represents the model's performance on a separate part of the dataset not seen during training (the validation set). It's crucial for assessing how well the model generalizes to new, unseen data. Like training loss, lower is better. The validation loss decreased from 4.671835 to 2.296098, indicating that the model's predictions became more accurate with respect to the validation set as training progressed.

**global_step**: The total number of training steps completed. A step is a single gradient update; this number is 51, indicating your model's weights were updated 51 times.

**training_loss:** The average loss over all training steps. Here, it's somewhat higher than your final validation loss, which is typical as the model learns from the training data.

**metrics:** This dictionary contains various performance metrics:

**train_runtime:** Total training time in seconds (605.114).

**train_samples_per_second:** The number of training samples processed per second (0.322).

**train_steps_per_second**:The number of training steps (weight updates) performed per second (0.084).

**total_flos:** The total number of floating-point operations performed during training. This gives an idea of the computational cost (26,391,651,287,040 FLOPs).

**train_loss:** Repetition of the training loss.

**epoch**: The final epoch number (3.0).